In [39]:
import datapreprocessing as dp
import matrix_factorization as mf
import numpy as np
import pandas as pd
import torch

In [8]:
df_Apps = dp.getDF("Apps_for_Android_5.json")
df_Apps.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1N4O8VOJZTDVB,B004A9SDD8,Annette Yancey,"[1, 1]","Loves the song, so he really couldn't wait to ...",3.0,Really cute,1383350400,"11 2, 2013"
1,A2HQWU6HUKIEC7,B004A9SDD8,"Audiobook lover ""Kathy""","[0, 0]","Oh, how my little grandson loves this app. He'...",5.0,2-year-old loves it,1323043200,"12 5, 2011"
2,A1SXASF6GYG96I,B004A9SDD8,Barbara Gibbs,"[0, 0]",I found this at a perfect time since my daught...,5.0,Fun game,1337558400,"05 21, 2012"
3,A2B54P9ZDYH167,B004A9SDD8,"Brooke Greenstreet ""Babylove""","[3, 4]",My 1 year old goes back to this game over and ...,5.0,We love our Monkeys!,1354752000,"12 6, 2012"
4,AFOFZDTX5UC6D,B004A9SDD8,C. Galindo,"[1, 1]",There are three different versions of the song...,5.0,This is my granddaughters favorite app on my K...,1391212800,"02 1, 2014"


In [52]:
# normalize into [0,1]
def normalize(ratings):
    ratings = ratings.copy()
    max_ratings = ratings.max()
    ratings = ratings*1.0/max_ratings
    return ratings

In [53]:
normalize(df_Apps['overall'])

0         0.6
1         1.0
2         1.0
3         1.0
4         1.0
5         1.0
6         1.0
7         1.0
8         1.0
9         1.0
10        0.8
11        0.6
12        1.0
13        0.6
14        1.0
15        1.0
16        0.8
17        0.2
18        1.0
19        0.8
20        0.8
21        1.0
22        1.0
23        1.0
24        0.2
25        1.0
26        0.2
27        1.0
28        1.0
29        1.0
         ... 
752907    1.0
752908    1.0
752909    0.8
752910    1.0
752911    1.0
752912    1.0
752913    1.0
752914    1.0
752915    1.0
752916    0.8
752917    1.0
752918    1.0
752919    1.0
752920    1.0
752921    0.8
752922    1.0
752923    1.0
752924    1.0
752925    1.0
752926    1.0
752927    1.0
752928    1.0
752929    1.0
752930    1.0
752931    1.0
752932    1.0
752933    0.6
752934    0.8
752935    1.0
752936    0.8
Name: overall, Length: 752937, dtype: float64

In [18]:
n_users = df_Apps.reviewerID.unique().shape[0]
print('{} unique users'.format(n_users))

87271 unique users


In [19]:
n_items = df_Apps.asin.unique().shape[0]
print('{} unique items'.format(n_items))

13209 unique items


rating matrix with one row per user and one column per movie using the 'pivot table' function

In [23]:
ratings = df_Apps.pivot(index='reviewerID', columns='asin', values='overall').fillna(0)

In [24]:
ratings.head(50)

asin,B004A9SDD8,B004AFQAUA,B004AHBBPW,B004ALVL6W,B004AMAIZQ,B004AMDC86,B004ANC00Q,B004ANE2WU,B004ANMWPY,B004AZH4C8,...,B00LDNE3FG,B00LDNQHJ6,B00LEIT6K2,B00LFLOEYG,B00LIAMYCI,B00LITLOMK,B00LMLW8T2,B00LMLXTJK,B00LP1MVSW,B00LUEMK44
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00100742Q4O8VH0YMUBZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A001619027H9L9EG4UVRB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A002359833QJM7OQHCXWY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0037670NPLI11RBWYFA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A003841815JTX0JFLR8B1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0038872349TB5N0JHQQW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00455683H6M1GQZMPQPV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0047670XAAJD587LXS7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00501041JRAPWYLPQ4TE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
ratings = np.array(ratings)
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0]*ratings.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))

Sparsity: 0.07%


In [27]:
model = mf.MatrixFactorization(n_users, n_items)

In [34]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
# Loss with L_2 regularization
#reg_loss_fn = torch.optim.SGD(model.parameters(), lr=1e-6,
#                             weight_decay=1e-5)

In [31]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

In [35]:
for row, col in zip(*(rows, cols)):
    # Turn data into variables
    rating = torch.FloatTensor([ratings[row, col]])
    row = torch.LongTensor([np.long(row)])
    col = torch.LongTensor([np.long(col)])

    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_fn(prediction, rating)

    # Backpropagate
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print
    print(loss)

tensor(66.0005, grad_fn=<MseLossBackward>)
tensor(1.9774, grad_fn=<MseLossBackward>)
tensor(91.6970, grad_fn=<MseLossBackward>)
tensor(14.4182, grad_fn=<MseLossBackward>)
tensor(52.4160, grad_fn=<MseLossBackward>)
tensor(13.7617, grad_fn=<MseLossBackward>)
tensor(31.6274, grad_fn=<MseLossBackward>)
tensor(98.2782, grad_fn=<MseLossBackward>)
tensor(1.6357, grad_fn=<MseLossBackward>)
tensor(3.7664e-06, grad_fn=<MseLossBackward>)
tensor(7.3301, grad_fn=<MseLossBackward>)
tensor(231.1281, grad_fn=<MseLossBackward>)
tensor(0.1855, grad_fn=<MseLossBackward>)
tensor(28.2006, grad_fn=<MseLossBackward>)
tensor(6.1128, grad_fn=<MseLossBackward>)
tensor(4.2869, grad_fn=<MseLossBackward>)
tensor(78.8055, grad_fn=<MseLossBackward>)
tensor(0.6758, grad_fn=<MseLossBackward>)
tensor(3.3179, grad_fn=<MseLossBackward>)
tensor(28.7021, grad_fn=<MseLossBackward>)
tensor(119.2211, grad_fn=<MseLossBackward>)
tensor(34.6445, grad_fn=<MseLossBackward>)
tensor(24.0494, grad_fn=<MseLossBackward>)
tensor(0.0003,

tensor(1.0877, grad_fn=<MseLossBackward>)
tensor(64.6558, grad_fn=<MseLossBackward>)
tensor(57.9609, grad_fn=<MseLossBackward>)
tensor(23.8666, grad_fn=<MseLossBackward>)
tensor(35.9404, grad_fn=<MseLossBackward>)
tensor(39.3976, grad_fn=<MseLossBackward>)
tensor(0.0817, grad_fn=<MseLossBackward>)
tensor(97.3150, grad_fn=<MseLossBackward>)
tensor(18.5254, grad_fn=<MseLossBackward>)
tensor(110.2711, grad_fn=<MseLossBackward>)
tensor(89.9900, grad_fn=<MseLossBackward>)
tensor(192.3101, grad_fn=<MseLossBackward>)
tensor(20.6011, grad_fn=<MseLossBackward>)
tensor(153.7101, grad_fn=<MseLossBackward>)
tensor(17.3270, grad_fn=<MseLossBackward>)
tensor(6.1410, grad_fn=<MseLossBackward>)
tensor(21.2268, grad_fn=<MseLossBackward>)
tensor(110.8274, grad_fn=<MseLossBackward>)
tensor(30.1041, grad_fn=<MseLossBackward>)
tensor(24.6666, grad_fn=<MseLossBackward>)
tensor(71.2665, grad_fn=<MseLossBackward>)
tensor(18.8087, grad_fn=<MseLossBackward>)
tensor(13.7015, grad_fn=<MseLossBackward>)
tensor(32.

tensor(25.9089, grad_fn=<MseLossBackward>)
tensor(129.8119, grad_fn=<MseLossBackward>)
tensor(50.8563, grad_fn=<MseLossBackward>)
tensor(24.4952, grad_fn=<MseLossBackward>)
tensor(86.9580, grad_fn=<MseLossBackward>)
tensor(183.4841, grad_fn=<MseLossBackward>)
tensor(2.9391, grad_fn=<MseLossBackward>)
tensor(44.7561, grad_fn=<MseLossBackward>)
tensor(115.7588, grad_fn=<MseLossBackward>)
tensor(0.2255, grad_fn=<MseLossBackward>)
tensor(6.6574, grad_fn=<MseLossBackward>)
tensor(6.1237, grad_fn=<MseLossBackward>)
tensor(20.5587, grad_fn=<MseLossBackward>)
tensor(62.7727, grad_fn=<MseLossBackward>)
tensor(52.6474, grad_fn=<MseLossBackward>)
tensor(15.3408, grad_fn=<MseLossBackward>)
tensor(22.0395, grad_fn=<MseLossBackward>)
tensor(45.6314, grad_fn=<MseLossBackward>)
tensor(1.3827, grad_fn=<MseLossBackward>)
tensor(35.2682, grad_fn=<MseLossBackward>)
tensor(0.8731, grad_fn=<MseLossBackward>)
tensor(6.9600, grad_fn=<MseLossBackward>)
tensor(2.3572, grad_fn=<MseLossBackward>)
tensor(28.9597, 

tensor(0.9197, grad_fn=<MseLossBackward>)
tensor(9.2860, grad_fn=<MseLossBackward>)
tensor(267.9572, grad_fn=<MseLossBackward>)
tensor(1.2816, grad_fn=<MseLossBackward>)
tensor(163.3722, grad_fn=<MseLossBackward>)
tensor(9.2134, grad_fn=<MseLossBackward>)
tensor(130.9089, grad_fn=<MseLossBackward>)
tensor(105.1622, grad_fn=<MseLossBackward>)
tensor(1.1226, grad_fn=<MseLossBackward>)
tensor(0.1311, grad_fn=<MseLossBackward>)
tensor(31.5539, grad_fn=<MseLossBackward>)
tensor(1.5337, grad_fn=<MseLossBackward>)
tensor(22.4960, grad_fn=<MseLossBackward>)
tensor(0.0834, grad_fn=<MseLossBackward>)
tensor(78.0278, grad_fn=<MseLossBackward>)
tensor(13.0246, grad_fn=<MseLossBackward>)
tensor(16.7137, grad_fn=<MseLossBackward>)
tensor(21.1753, grad_fn=<MseLossBackward>)
tensor(21.3095, grad_fn=<MseLossBackward>)
tensor(11.0882, grad_fn=<MseLossBackward>)
tensor(53.3434, grad_fn=<MseLossBackward>)
tensor(4.7993, grad_fn=<MseLossBackward>)
tensor(77.0542, grad_fn=<MseLossBackward>)
tensor(66.9947, 

tensor(0.5930, grad_fn=<MseLossBackward>)
tensor(3.1097, grad_fn=<MseLossBackward>)
tensor(74.0297, grad_fn=<MseLossBackward>)
tensor(0.1099, grad_fn=<MseLossBackward>)
tensor(12.8405, grad_fn=<MseLossBackward>)
tensor(5.7500, grad_fn=<MseLossBackward>)
tensor(0.0578, grad_fn=<MseLossBackward>)
tensor(0.0146, grad_fn=<MseLossBackward>)
tensor(28.0621, grad_fn=<MseLossBackward>)
tensor(1.3933, grad_fn=<MseLossBackward>)
tensor(0.1036, grad_fn=<MseLossBackward>)
tensor(59.6628, grad_fn=<MseLossBackward>)
tensor(82.5612, grad_fn=<MseLossBackward>)
tensor(25.1258, grad_fn=<MseLossBackward>)
tensor(16.9577, grad_fn=<MseLossBackward>)
tensor(46.3785, grad_fn=<MseLossBackward>)
tensor(51.6016, grad_fn=<MseLossBackward>)
tensor(11.9136, grad_fn=<MseLossBackward>)
tensor(26.2670, grad_fn=<MseLossBackward>)
tensor(145.0914, grad_fn=<MseLossBackward>)
tensor(59.5105, grad_fn=<MseLossBackward>)
tensor(78.2468, grad_fn=<MseLossBackward>)
tensor(47.7923, grad_fn=<MseLossBackward>)
tensor(39.1068, gr

tensor(16.9239, grad_fn=<MseLossBackward>)
tensor(0.6248, grad_fn=<MseLossBackward>)
tensor(7.9091, grad_fn=<MseLossBackward>)
tensor(11.1115, grad_fn=<MseLossBackward>)
tensor(51.3012, grad_fn=<MseLossBackward>)
tensor(2.3860, grad_fn=<MseLossBackward>)
tensor(14.4048, grad_fn=<MseLossBackward>)
tensor(208.6924, grad_fn=<MseLossBackward>)
tensor(9.0940, grad_fn=<MseLossBackward>)
tensor(16.1395, grad_fn=<MseLossBackward>)
tensor(1.9030, grad_fn=<MseLossBackward>)
tensor(155.0233, grad_fn=<MseLossBackward>)
tensor(92.1983, grad_fn=<MseLossBackward>)
tensor(0.0002, grad_fn=<MseLossBackward>)
tensor(0.2920, grad_fn=<MseLossBackward>)
tensor(22.7661, grad_fn=<MseLossBackward>)
tensor(72.5573, grad_fn=<MseLossBackward>)
tensor(4.0176, grad_fn=<MseLossBackward>)
tensor(130.8514, grad_fn=<MseLossBackward>)
tensor(26.9604, grad_fn=<MseLossBackward>)
tensor(1.5139, grad_fn=<MseLossBackward>)
tensor(120.5813, grad_fn=<MseLossBackward>)
tensor(0.4415, grad_fn=<MseLossBackward>)
tensor(24.0252, g

tensor(2.8330, grad_fn=<MseLossBackward>)
tensor(1.0436, grad_fn=<MseLossBackward>)
tensor(45.7557, grad_fn=<MseLossBackward>)
tensor(8.0123, grad_fn=<MseLossBackward>)
tensor(5.2081, grad_fn=<MseLossBackward>)
tensor(38.0729, grad_fn=<MseLossBackward>)
tensor(27.9268, grad_fn=<MseLossBackward>)
tensor(0.8369, grad_fn=<MseLossBackward>)
tensor(31.3238, grad_fn=<MseLossBackward>)
tensor(12.5222, grad_fn=<MseLossBackward>)
tensor(2.8614, grad_fn=<MseLossBackward>)
tensor(80.3654, grad_fn=<MseLossBackward>)
tensor(13.8345, grad_fn=<MseLossBackward>)
tensor(13.7441, grad_fn=<MseLossBackward>)
tensor(53.7577, grad_fn=<MseLossBackward>)
tensor(27.1640, grad_fn=<MseLossBackward>)
tensor(30.2812, grad_fn=<MseLossBackward>)
tensor(140.2981, grad_fn=<MseLossBackward>)
tensor(240.3742, grad_fn=<MseLossBackward>)
tensor(173.4011, grad_fn=<MseLossBackward>)
tensor(13.2317, grad_fn=<MseLossBackward>)
tensor(23.4730, grad_fn=<MseLossBackward>)
tensor(122.1574, grad_fn=<MseLossBackward>)
tensor(6.0992

KeyboardInterrupt: 